# Dialogue Summarization: Baseline vs System Prompt

This notebook compares Llama's performance on dialogue summarization with and without a system prompt. We'll evaluate using ROUGE scores to measure summary quality.


In [1]:
! pip install -q torch datasets peft huggingface_hub evaluate rouge_score hf_transfer

In [37]:
import os
import warnings
import torch
import evaluate
from tqdm import tqdm
from typing import Optional
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline
)
from huggingface_hub import login
from datasets import Dataset, load_dataset
import evaluate
from tqdm.auto import tqdm
from pprint import pprint

warnings.filterwarnings("ignore")

## 2. Authentication and Configuration

Before we begin training, we need to authenticate with HuggingFace (to download models and upload results) and optionally with Weights & Biases (to track training progress).

**Important:** Make sure you have a `.env` file with:
- `HF_TOKEN`: Your HuggingFace access token
- `HF_USERNAME`: Your HuggingFace username

We'll also load our training configuration from `config.json`, which contains all hyperparameters and settings.


## Load Data from HuggingFace

In [3]:
dataset = load_dataset("knkarthick/samsum")
train_data = dataset["train"]

test_data = dataset['test'].shuffle(seed=42).select(range(200))
val_data = dataset['validation'].shuffle(seed=42).select(range(200))

README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/14731 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

## Helper Methods for Evaluation

In [4]:
def build_summary_prompt(sample: dict, with_answer: bool = False, system_prompt: str = None) -> str:
    """
    Build a formatted summarization prompt from a single dataset sample.

    Args:
        sample (dict): Dictionary containing 'dialogue' and 'summary' keys.
        with_answer (bool, optional): If True, include the ground truth summary (for training). Defaults to False.
        system_prompt (str, optional): Optional system-level instruction to prepend (e.g. "You are a helpful assistant.").

    Returns:
        str: Formatted prompt string ready for model input.
    """
    dialogue = sample.get("dialogue", "").strip()
    summary = sample.get("summary", "").strip()

    # Optional system-level prefix
    sys_part = f"{system_prompt.strip()}\n\n" if system_prompt else ""

    # Build the base summarization prompt
    prompt = (
        f"{sys_part}"
        f"## Dialogue:\n{dialogue}\n## Summary:"
    )

    # Add ground truth summary if needed (for training)
    if with_answer and summary:
        prompt += f" {summary}"

    return prompt


In [5]:
sample = val_data[0]
print("Prompt without system prompt.")
print(build_summary_prompt(sample))


print("-"*60)
print("Prompt with system prompt.")
system_prompt = (
    "You are a concise, factual summarization assistant. Summarize the following conversation into a single sentence." 
)
print(build_summary_prompt(sample, system_prompt=system_prompt))

Prompt without system prompt.
## Dialogue:
Victoria: God I'm really broke, I spent way to much this month 😫
Victoria: At least we get paid soon..
Magda: Yeah, don't remind me, I know the feeling
Magda: I just paid my car insurance, I feel robbed 😂
Victoria: Thankfully mine is paid for the rest of the year 🙏
Magda: 👌
## Summary:
------------------------------------------------------------
Prompt with system prompt.
You are a concise, factual summarization assistant. Summarize the following conversation into a single sentence.

## Dialogue:
Victoria: God I'm really broke, I spent way to much this month 😫
Victoria: At least we get paid soon..
Magda: Yeah, don't remind me, I know the feeling
Magda: I just paid my car insurance, I feel robbed 😂
Victoria: Thankfully mine is paid for the rest of the year 🙏
Magda: 👌
## Summary:


# Evaluation Function

In [11]:
rouge = evaluate.load("rouge")

def evaluate_model(pipe, samples, num_samples=None, system_prompt=None, batch_size=4):
    """
    Evaluate summarization performance on a subset of samples using ROUGE metrics.
    Processes samples in batches for better efficiency.
    """
    from tqdm.auto import tqdm
    from datasets import Dataset
    
    # Apply num_samples limit safely
    if num_samples is not None and num_samples < len(samples):
        eval_data = samples.select(range(num_samples))
        print(f"Evaluating a subset of {num_samples} samples...")
    else:
        eval_data = samples
        print(f"Evaluating all {len(samples)} samples...")
    
    # Prepare prompts and ground truth summaries
    prompts = [
        build_summary_prompt(ex, system_prompt=system_prompt, with_answer=False)
        for ex in eval_data
    ]
    references = [ex["summary"] for ex in eval_data]
    
    # Convert to Hugging Face Dataset for efficient batching
    dataset = Dataset.from_dict({"prompt": prompts})
        
    predictions = []
    total_samples = len(dataset)
    
    # Run inference in batches with progress bar
    pbar = tqdm(total=total_samples, desc="Generating summaries", leave=True, position=0)
    
    for start in range(0, total_samples, batch_size):
        end = min(start + batch_size, total_samples)
        batch_prompts = dataset["prompt"][start:end]
        
        # Generate summaries for the batch
        outputs = pipe(batch_prompts, max_new_tokens=256, batch_size=batch_size, return_full_text=False)
        
        for output in outputs:
            generated_text = output[0]["generated_text"].strip()
            predictions.append(generated_text)
            pbar.update(1)
    
    pbar.close()
    
    # Compute ROUGE scores using the rouge object directly
    print("\nComputing ROUGE scores...")
    results = rouge.compute(predictions=predictions, references=references)
    
    return results

## Load the Model

In [9]:
model_name = "meta-llama/Llama-3.2-1B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    dtype=torch.bfloat16,
)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

## Baseline Evaluation (No System Prompt)
First, we evaluate the model without any system prompt. This gives us a baseline performance to compare against.


In [10]:
pipe = pipeline(
    "text-generation",
    model=base_model,
    tokenizer=tokenizer,
    device_map="auto",
    temperature=0.01,
    do_sample=False
)

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [12]:
rouge_scores_no_sys_prompt = evaluate_model(
    pipe=pipe,
    samples=val_data,
    system_prompt=None,
    # num_samples=8,
)

print("ROUGE SCORES WITHOUT SYSTEM PROMPT")
print(f'rouge1: {rouge_scores_no_sys_prompt['rouge1']:.2f}')
print(f'rouge2: {rouge_scores_no_sys_prompt['rouge2']:.2f}')
print(f'rougeL: {rouge_scores_no_sys_prompt['rougeL']:.2f}')
print(f'rougeLsum: {rouge_scores_no_sys_prompt['rougeLsum']:.2f}')

torch.cuda.empty_cache()

Evaluating all 200 samples...


Generating summaries:   0%|          | 0/200 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



Computing ROUGE scores...
rouge1: 0.18
rouge2: 0.06
rougeL: 0.14
rougeLsum: 0.14


## Evaluation with System Prompt

Now we evaluate the same model with a system prompt that provides specific instructions for summarization. This helps the model understand the task better and should improve performance.


In [18]:
system_prompt = (
    "You are a helpful assistant who writes concise, factual summaries of conversations. "
    "Summarize the following conversation into a single sentence."
)

rouge_scores_w_sys_prompt = evaluate_model(
    pipe=pipe,
    samples=val_data,
    system_prompt=system_prompt,
    # num_samples=200,
)

print("ROUGE SCORES WITH SYSTEM PROMPT")
print(f'rouge1: {rouge_scores_w_sys_prompt['rouge1']:.2f}')
print(f'rouge2: {rouge_scores_w_sys_prompt['rouge2']:.2f}')
print(f'rougeL: {rouge_scores_w_sys_prompt['rougeL']:.2f}')
print(f'rougeLsum: {rouge_scores_w_sys_prompt['rougeLsum']:.2f}')

torch.cuda.empty_cache()

Evaluating all 200 samples...


Generating summaries:   0%|          | 0/200 [00:00<?, ?it/s]


Computing ROUGE scores...
ROUGE SCORES WITH SYSTEM PROMPT
rouge1: 0.35
rouge2: 0.13
rougeL: 0.27
rougeLsum: 0.27


## Example Generation

Let's look at a specific example to see how the model performs with and without the system prompt.


In [38]:
# Pick a sample
sample = val_data[0]
pprint(sample)

{'dialogue': "Victoria: God I'm really broke, I spent way to much this month "
             '😫\n'
             'Victoria: At least we get paid soon..\n'
             "Magda: Yeah, don't remind me, I know the feeling\n"
             'Magda: I just paid my car insurance, I feel robbed 😂\n'
             'Victoria: Thankfully mine is paid for the rest of the year 🙏\n'
             'Magda: 👌',
 'id': '13829853',
 'summary': 'Magda and Victoria feel broke. '}


In [40]:
# Summary without system prompt

print("-"*60)
sample_prompt_no_system_prompt = build_summary_prompt(sample, with_answer=False)
pprint(f"sample_prompt_no_system_prompt=\n{sample_prompt_no_system_prompt}")
response = pipe(sample_prompt_no_system_prompt, max_new_tokens=256, return_full_text=False)
print("-"*60)
print("Generated Summary without system prompt =")
print(response[0]['generated_text'])
print("-"*60)

------------------------------------------------------------
('sample_prompt_no_system_prompt=\n'
 '## Dialogue:\n'
 "Victoria: God I'm really broke, I spent way to much this month 😫\n"
 'Victoria: At least we get paid soon..\n'
 "Magda: Yeah, don't remind me, I know the feeling\n"
 'Magda: I just paid my car insurance, I feel robbed 😂\n'
 'Victoria: Thankfully mine is paid for the rest of the year 🙏\n'
 'Magda: 👌\n'
 '## Summary:')
------------------------------------------------------------
Generated Summary without system prompt =
 
Victoria and Magda are discussing their financial situation. Victoria is feeling broke and has spent a lot of money this month. Magda is also feeling financially strained, having paid her car insurance, but is relieved that her car is paid for the rest of the year. They both share a laugh at Magda's joke about being "robbed" of her car insurance payment.
------------------------------------------------------------


In [41]:
# Summary with system prompt

print("-"*60)
sample_with_system_prompt = build_summary_prompt(sample, system_prompt=system_prompt, with_answer=False)
pprint(f"sample_with_system_prompt=\n{sample_with_system_prompt}")
response = pipe(sample_with_system_prompt, max_new_tokens=256, return_full_text=False)
print("-"*60)
print("Generated Summary with system prompt =")
print(response[0]['generated_text'])
print("-"*60)

------------------------------------------------------------
('sample_with_system_prompt=\n'
 'You are a helpful assistant who writes concise, factual summaries of '
 'conversations. Summarize the following conversation into a single sentence.\n'
 '\n'
 '## Dialogue:\n'
 "Victoria: God I'm really broke, I spent way to much this month 😫\n"
 'Victoria: At least we get paid soon..\n'
 "Magda: Yeah, don't remind me, I know the feeling\n"
 'Magda: I just paid my car insurance, I feel robbed 😂\n'
 'Victoria: Thankfully mine is paid for the rest of the year 🙏\n'
 'Magda: 👌\n'
 '## Summary:')
------------------------------------------------------------
Generated Summary with system prompt =
 Victoria and Magda discuss their financial struggles, with Victoria expressing frustration about overspending and Magda joking about being "robbed" of their car insurance payments.
------------------------------------------------------------
